In [ ]:
import pandas as pd 
import numpy as np


In [3]:
df = pd.read_csv('wimbeldon.csv')
df = df[['PlayerA', 'PlayerB', 'RankA', 'RankB', 'WinnerBinary']]
df.head()

,PlayerA,PlayerB,RankA,RankB,WinnerBinary
0,Alcaraz C.,Djokovic N.,3,2,1
1,Djokovic N.,Musetti L.,2,25,1
2,Alcaraz C.,Medvedev D.,3,5,1
3,Fritz T.,Musetti L.,12,25,0
4,De Minaur A.,Djokovic N.,9,2,0


## Afegir noves columnes
#### RankDiff
Aquest camp serà la diferencia de rang que té el jugador A contra el B. Si el valor és positiu vol dir que el jugador A és X posicions millor que el B.


**RankDiff = RankB-RankB**

In [4]:
df['RankDiff'] = df['RankB'] - df['RankA']
df.head()

,PlayerA,PlayerB,RankA,RankB,WinnerBinary,RankDiff
0,Alcaraz C.,Djokovic N.,3,2,1,-1
1,Djokovic N.,Musetti L.,2,25,1,23
2,Alcaraz C.,Medvedev D.,3,5,1,2
3,Fritz T.,Musetti L.,12,25,0,13
4,De Minaur A.,Djokovic N.,9,2,0,-7


Fem una petita prova, per veure que tal va el model de regressio amb aquests camps.

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

lr = LogisticRegression()

X = df[['RankA', 'RankB', 'RankDiff']]
y = df['WinnerBinary']

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report

print("Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy:  0.7007874015748031
              precision    recall  f1-score   support

           0       0.72      0.66      0.69       383
           1       0.68      0.74      0.71       379

    accuracy                           0.70       762
   macro avg       0.70      0.70      0.70       762
weighted avg       0.70      0.70      0.70       762



Provem ara a ficar el nom a veure si fa alguna cosa.

In [17]:
lr_aux = LogisticRegression()

X = pd.get_dummies(
    df[['RankA', 'RankB', 'RankDiff', 'PlayerA', 'PlayerB']],
    columns=['PlayerA', 'PlayerB'],
    drop_first=True
)

y = df['WinnerBinary']

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

lr_aux.fit(X_train, y_train)

y_pred = lr_aux.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report

print("Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy:  0.6706036745406824
              precision    recall  f1-score   support

           0       0.64      0.69      0.66       361
           1       0.70      0.65      0.68       401

    accuracy                           0.67       762
   macro avg       0.67      0.67      0.67       762
weighted avg       0.67      0.67      0.67       762



#### PlayerA Odds i PlayerB Odds
Aquests camps són la mitja de les apostes sobre cada jugados abans de jugar el partit.

In [43]:
df_bets = pd.read_csv('wimbeldon_bets.csv')

# Afegim el RankDiff
df_bets['RankDiff'] = df_bets['RankB'] - df_bets['RankA']

df_bets.head()

,PlayerA,PlayerB,RankA,RankB,WinnerBinary,MaxOddsPlayerA,MaxOddsPlayerB,AvgOddsPlayerA,AvgOddsPlayerB,RankDiff
0,Alcaraz C.,Djokovic N.,3,2,1,1.82,2.20,1.76,2.09,-1
1,Djokovic N.,Musetti L.,2,25,1,1.15,6.75,1.12,6.30,23
2,Alcaraz C.,Medvedev D.,3,5,1,1.31,3.90,1.29,3.62,2
3,Fritz T.,Musetti L.,12,25,0,1.25,4.70,1.22,4.36,13
4,De Minaur A.,Djokovic N.,9,2,0,4.00,1.34,3.76,1.28,-7


Comencem a fer la regressió logistica

In [44]:
# Creem una nova regressió logistica
lr_bets = LogisticRegression()

# Eliminem nans
df_bets = df_bets.dropna(subset=['RankA', 'RankB', 'RankDiff', 'MaxOddsPlayerA', 'MaxOddsPlayerB', 'AvgOddsPlayerA', 'AvgOddsPlayerB', 'WinnerBinary'])

# Agafem els camps que necesitem per fer la predicció
X = df_bets[['RankA', 'RankB', 'RankDiff', 'MaxOddsPlayerA', 'MaxOddsPlayerB', 'AvgOddsPlayerA', 'AvgOddsPlayerB']]
y = df_bets['WinnerBinary']

# Dividim en test i train
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Escalem les X
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fem fit al model
lr_bets.fit(X_train, y_train)

# Extreiem la y de la predicció
y_pred = lr_bets.predict(X_test)

# Comparem la y de la predicció amb la test
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)
print(classification_report(y_test, y_pred))

Accuracy:  0.7752808988764045
              precision    recall  f1-score   support

           0       0.78      0.78      0.78       225
           1       0.78      0.77      0.77       220

    accuracy                           0.78       445
   macro avg       0.78      0.78      0.78       445
weighted avg       0.78      0.78      0.78       445



Vale, ara tenim un **>75%**, està molt bé, anem a afegir un parell més de dades!

In [45]:
df_bets['OddsDiff'] = df_bets['AvgOddsPlayerB'] - df_bets['AvgOddsPlayerA']
df_bets['OddsRatio'] = df_bets['AvgOddsPlayerA'] / df_bets['AvgOddsPlayerB']

Hem afegit la diferencia i el ratio sobre les apostes d'abans del partit, millorarà el model?

In [ ]:
# Creem una nova regressió logistica
lr_bets_2 = LogisticRegression()

# Eliminem nans
df_bets = df_bets.dropna(subset=['RankA', 'RankB', 'RankDiff', 'MaxOddsPlayerA', 'MaxOddsPlayerB', 'AvgOddsPlayerA', 'AvgOddsPlayerB', 'WinnerBinary', 'OddsDiff', 'OddsRatio'])

# Agafem els camps que necesitem per fer la predicció
X = df_bets[['RankA', 'RankB', 'RankDiff', 'MaxOddsPlayerA', 'MaxOddsPlayerB', 'AvgOddsPlayerA', 'AvgOddsPlayerB', 'OddsDiff', 'OddsRatio']]
y = df_bets['WinnerBinary']

# Dividim en test i train
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Escalem les X
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fem fit al model
lr_bets_2.fit(X_train, y_train)

# Extreiem la y de la predicció
y_pred = lr_bets_2.predict(X_test)

# Comparem la y de la predicció amb la test
accuracy_2 = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy_2)
print(classification_report(y_test, y_pred))

# Comparem la millora amb l'anterior
print("Millora respecte anterior: ", ((accuracy_2-accuracy)/accuracy)*100, "%")

Accuracy:  0.7820224719101123
              precision    recall  f1-score   support

           0       0.80      0.79      0.79       235
           1       0.77      0.78      0.77       210

    accuracy                           0.78       445
   macro avg       0.78      0.78      0.78       445
weighted avg       0.78      0.78      0.78       445

Millora respecte anterior:  0.869565217391298  %


Com podem veure, no hi ha molta millora, seguim provant.

#### H2H
En aquest cas, farem 3 noves columnes:
- **H2H_A_Wins**: Cuants cops el jugador A ha guanyat al jugador B
- **H2H_B_Wins**: Cuants cops el jugador B ha guanyat al jugador A
- **H2H_Diff**: Si A ha guamyat més a B, número positu, en cas contrari número negatiu. En cas d'empat, 0.

In [68]:
df_h2h = pd.read_csv('wimbeldon_bets_h2h.csv')

# Afegim el RankDiff
df_h2h['RankDiff'] = df_h2h['RankB'] - df_h2h['RankA']

# Afegim la diferencia de odds
df_h2h['OddsDiff'] = df_h2h['AvgOddsPlayerB'] - df_h2h['AvgOddsPlayerA']
df_h2h['OddsRatio'] = df_h2h['AvgOddsPlayerA'] / df_h2h['AvgOddsPlayerB']

df_h2h.head()

# Descarreguem aquesta taula a un .csv
df_h2h.to_csv('final.csv', index=False)

Ara que ya tenim tot preparat, provem a veure que ens surt!

In [ ]:
# Creem una nova regressió logistica
lr_h2h = LogisticRegression()

# Eliminem nans
df_h2h = df_h2h.dropna(subset=['RankA', 'RankB', 'RankDiff', 'MaxOddsPlayerA', 'MaxOddsPlayerB', 'AvgOddsPlayerA', 'AvgOddsPlayerB', 'WinnerBinary', 'OddsDiff', 'OddsRatio', 'H2H_A_wins', 'H2H_B_wins', 'H2H_Diff'])

# Agafem els camps que necesitem per fer la predicció
X = df_h2h[['RankA', 'RankB', 'RankDiff', 'MaxOddsPlayerA', 'MaxOddsPlayerB', 'AvgOddsPlayerA', 'AvgOddsPlayerB', 'OddsDiff', 'OddsRatio', 'H2H_A_wins', 'H2H_B_wins', 'H2H_Diff']]
y = df_h2h['WinnerBinary']

# Dividim en test i train
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Escalem les X
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fem fit al model
lr_h2h.fit(X_train, y_train)

# Extreiem la y de la predicció
y_pred = lr_h2h.predict(X_test)

# Comparem la y de la predicció amb la test
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)
print(classification_report(y_test, y_pred))


Accuracy:  0.851685393258427
              precision    recall  f1-score   support

           0       0.87      0.85      0.86       235
           1       0.84      0.85      0.84       210

    accuracy                           0.85       445
   macro avg       0.85      0.85      0.85       445
weighted avg       0.85      0.85      0.85       445



PERFECTE, tenim un 85%!